## wan transformer 3D model

In [1]:
import sys
import traceback

from finetrainers import BaseArgs, ControlTrainer, SFTTrainer, TrainingType, get_logger
from finetrainers.config import _get_model_specifiction_cls
from finetrainers.trainer.control_trainer.config import ControlFullRankConfig, ControlLowRankConfig
from finetrainers.trainer.sft_trainer.config import SFTFullRankConfig, SFTLowRankConfig
logger = get_logger()


/home/wjh/miniconda3/envs/finetrainer/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [25]:
model_name = 'wan'
training_type = 'lora'  # 'control-lora'
# model: finetrainers/models/wan/control_specification.py:l36 WanControlModelSpecification
# which include: LoRA+control
# ---
# 其实可能不需要用control branch
# /home/wjh/projects/finetrainers/finetrainers/models/wan/base_specification.py WanModelSpecification
model_specification_cls = _get_model_specifiction_cls(model_name, training_type)

In [26]:
# args
args = BaseArgs()
training_cls = ControlLowRankConfig # TrainingType.CONTROL_LORA
args.register_args(training_cls())
t2v = True # 1.3B T2V model
if t2v:
	args.pretrained_model_name_or_path = 'Wan-AI/Wan2.1-T2V-1.3B-Diffusers' # T2V
else: 
    # Wan2.1-I2V-14B-720P
    # Wan2.1-T2V-14B-480P
	args.pretrained_model_name_or_path = 'Wan-AI/Wan2.1-1.3B-Diffusers'
load_local = True # load local model
if load_local:
    # load local model
	args.pretrained_model_name_or_path = '/home/wjh/projects/finetrainers/finetrainers_ckpts/Wan2.1-T2V-1.3B-Diffusers'
# ------- initialize args -------
model_specification = model_specification_cls(
		pretrained_model_name_or_path=args.pretrained_model_name_or_path,
		tokenizer_id=args.tokenizer_id,
		tokenizer_2_id=args.tokenizer_2_id,
		tokenizer_3_id=args.tokenizer_3_id,
		text_encoder_id=args.text_encoder_id,
		text_encoder_2_id=args.text_encoder_2_id,
		text_encoder_3_id=args.text_encoder_3_id,
		transformer_id=args.transformer_id,
		vae_id=args.vae_id,
		text_encoder_dtype=args.text_encoder_dtype,
		text_encoder_2_dtype=args.text_encoder_2_dtype,
		text_encoder_3_dtype=args.text_encoder_3_dtype,
		transformer_dtype=args.transformer_dtype,
		vae_dtype=args.vae_dtype,
		revision=args.revision,
		cache_dir=args.cache_dir,
	)

In [ ]:
from diffusers import WanTransformer3DModel
# load model
diffusion_components = model_specification.load_diffusion_models()
transformer = diffusion_components['transformer'].to('cuda:0')
schedule = diffusion_components['scheduler']
transformer.state_dict


Loading checkpoint shards: 100%|██████████| 2/2 [01:00<00:00, 30.03s/it]


## trainer for finetuning the model

In [ ]:
args = BaseArgs()

argv = [y.strip() for x in sys.argv for y in x.split()]
training_type_index = argv.index("--training_type")
if training_type_index == -1:
	raise ValueError("Training type not provided in command line arguments.")

training_type = argv[training_type_index + 1]
# training_cls = None
# if training_type == TrainingType.LORA:
# 	training_cls = SFTLowRankConfig
# elif training_type == TrainingType.FULL_FINETUNE:
# 	training_cls = SFTFullRankConfig
# elif training_type == TrainingType.CONTROL_LORA:
# 	training_cls = ControlLowRankConfig
# elif training_type == TrainingType.CONTROL_FULL_FINETUNE:
# 	training_cls = ControlFullRankConfig
# else:
# 	raise ValueError(f"Training type {training_type} not supported.")
training_cls = SFTLowRankConfig
args.register_args(training_cls())
args = args.parse_args()
# [wjh wan2.1] load model specification class based on model name and training type [control-lora]
model_specification_cls = _get_model_specifiction_cls(args.model_name, args.training_type)


In [ ]:
path = '/home/wjh/projects/finetrainers/datasets/crush-smol/videos/1gGQy4nxyUo-Scene-016.mp4'
import av
av.cdcd ExceptionGroup

/home/wjh/miniconda3/envs/finetrainer/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'finetrainers.utils.video_utils'

In [ ]:
from diffusers import WanTransformer3DModel